# FUNCTION

In [1]:
import pandas as pd
from io import StringIO
from google.cloud import storage
import requests
from vertexai.generative_models import (
    GenerativeModel,
    GenerationConfig,
    HarmCategory,
    HarmBlockThreshold
)

def load_transcription_csv(bucket_name, file_path):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_path)
    content = blob.download_as_text()

    return pd.read_csv(StringIO(content))[['speaker', 'transcript']]

def get_prompt(**kwargs):
    with open('./prompt/disposition_code_cleaned_layer_1.txt', 'r') as file:
        prompt_template = file.read()

    return prompt_template.format(**kwargs)

def generate_prompt(bucket_name, file_path, interaction_id: str, list_of_option):
    transcript_df = load_transcription_csv(bucket_name,f"{file_path}/transcription/{interaction_id}/transcription_results.csv")
    transcript_df = transcript_df.where(pd.notnull(transcript_df), '')
    json_dict = transcript_df.to_dict(orient='records')
    prompt = get_prompt(
        transcript=json_dict,
        list_of_option=','.join(list_of_option),
    )
    return prompt

## MODEL LLM

In [2]:
model = GenerativeModel('gemini-1.5-flash-002')
generation_config = GenerationConfig(
    temperature=0.1,  # Example temperature value
    top_k=1,         # Example top-k value
    top_p=0.1        # Example top-p value
)

def generate_content(prompt):
    result = model.generate_content(
        contents=prompt,
        safety_settings={
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        },
        generation_config=generation_config
    )
    return result

In [3]:
import json

with open('./ref_data/list_of_option.json') as f:
    list_of_option = json.load(f)['value']

list_of_unknown_csv = [9151919416110000090, 9151919416110000091, 9151919416110000092]
output_data = []

for interaction_id in range(9151919416110000051, 9151919416110000100):
    if interaction_id in list_of_unknown_csv:
        continue
    try:
        prompt = generate_prompt('athena-nonprod-gcs', 'athena-ws4a/athena-qapm', interaction_id, list_of_option)
        response = generate_content(prompt).text
        disposition_result = json.loads(response.replace("```json", "").replace("```", "").strip())
        output_data.append({
            'interaction_id': interaction_id,
            'option_1': disposition_result[0]['value'],
            'explanation_1': disposition_result[0]['penjelasan'],
            'option_2': disposition_result[1]['value'] if len(disposition_result) > 1 else None,
            'explanation_2': disposition_result[1]['penjelasan'] if len(disposition_result) > 1 else None,
            'option_3': disposition_result[2]['value'] if len(disposition_result) > 2 else None,
            'explanation_3': disposition_result[2]['penjelasan'] if len(disposition_result) > 2 else None,
        })

        print(f"Processing interaction_id: {interaction_id}")
    except Exception as e:
        print(f"Error: {e} at interaction_id: {interaction_id}")

Processing interaction_id: 9151919416110000051
Processing interaction_id: 9151919416110000052
Processing interaction_id: 9151919416110000053
Processing interaction_id: 9151919416110000054
Processing interaction_id: 9151919416110000055
Processing interaction_id: 9151919416110000056
Processing interaction_id: 9151919416110000057
Processing interaction_id: 9151919416110000058
Processing interaction_id: 9151919416110000059


KeyboardInterrupt: 

In [13]:
df = pd.DataFrame(output_data)
df

,interaction_id,option_1,explanation_1,option_2,explanation_2,option_3,explanation_3
0,9151919416110000051,Complain,Nasabah mengeluhkan transaksi tarik tunai kart...,None,None,None,None
1,9151919416110000052,Complain,Nasabah melaporkan transaksi kartu kredit yang...,None,None,None,None
2,9151919416110000053,Complain,Nasabah mengeluhkan kegagalan transaksi GoPay ...,None,None,None,None
3,9151919416110000054,Complain,Nasabah mengeluhkan adanya transaksi ganda (do...,None,None,None,None
4,9151919416110000055,Complain,Nasabah mengeluhkan ketidakmampuan login ke Oc...,None,None,None,None
5,9151919416110000056,Inquiry,Nasabah menanyakan detail biaya-biaya yang mun...,None,None,None,None
6,9151919416110000057,Inquiry,Nasabah menanyakan sisa tagihan KPR-nya. Tran...,None,None,None,None
7,9151919416110000058,Inquiry,Nasabah menanyakan sisa limit kartu kredit dan...,None,None,None,None
8,9151919416110000059,Complain,Nasabah mengeluhkan pembayaran kartu kredit ya...,None,None,None,None
9,9151919416110000060,Inquiry,Nasabah menanyakan status penghapusan biaya an...,None,None,None,None


# Combine data with ground truth

In [14]:
# read json file
import json

with open('./ground_truth/ground_truth.json') as f:
    ground_truth = json.load(f)

# map with ground truth
df['interaction_id'] = df['interaction_id'].astype(str)
df['ground_truth'] = df['interaction_id'].map(ground_truth)

# save to csv
df.to_csv('output.csv', index=False)

# Read the output.csv file

In [24]:
df = pd.read_csv('output.csv')

# check whether option_1 is equal to ground_truth, the result is True or False in "is_match" column
df['is_match'] = df['ground_truth'] == df['option_1']
df_view = df[['interaction_id', 'option_1', 'ground_truth', 'is_match']]
df_view

,interaction_id,option_1,ground_truth,is_match
0,9151919416110000051,Complain,Complain,True
1,9151919416110000052,Complain,Complain,True
2,9151919416110000053,Complain,Complain,True
3,9151919416110000054,Complain,Complain,True
4,9151919416110000055,Complain,Complain,True
5,9151919416110000056,Inquiry,Inquiry,True
6,9151919416110000057,Inquiry,Inquiry,True
7,9151919416110000058,Inquiry,Inquiry,True
8,9151919416110000059,Complain,Inquiry,False
9,9151919416110000060,Inquiry,NaN,False


# CALCULATE ACCURACY

In [25]:
# calculate the mean of "is_match" column where ground_truth is not NaN
accuracy = df[df['ground_truth'].notnull()]['is_match'].mean()
accuracy

np.float64(0.8108108108108109)

In [20]:
df

,interaction_id,option_1,explanation_1,option_2,explanation_2,option_3,explanation_3,ground_truth,is_match
0,9151919416110000051,Complain,Nasabah mengeluhkan transaksi tarik tunai kart...,NaN,NaN,NaN,NaN,Complain,True
1,9151919416110000052,Complain,Nasabah melaporkan transaksi kartu kredit yang...,NaN,NaN,NaN,NaN,Complain,True
2,9151919416110000053,Complain,Nasabah mengeluhkan kegagalan transaksi GoPay ...,NaN,NaN,NaN,NaN,Complain,True
3,9151919416110000054,Complain,Nasabah mengeluhkan adanya transaksi ganda (do...,NaN,NaN,NaN,NaN,Complain,True
4,9151919416110000055,Complain,Nasabah mengeluhkan ketidakmampuan login ke Oc...,NaN,NaN,NaN,NaN,Complain,True
5,9151919416110000056,Inquiry,Nasabah menanyakan detail biaya-biaya yang mun...,NaN,NaN,NaN,NaN,Inquiry,True
6,9151919416110000057,Inquiry,Nasabah menanyakan sisa tagihan KPR-nya. Tran...,NaN,NaN,NaN,NaN,Inquiry,True
7,9151919416110000058,Inquiry,Nasabah menanyakan sisa limit kartu kredit dan...,NaN,NaN,NaN,NaN,Inquiry,True
8,9151919416110000059,Complain,Nasabah mengeluhkan pembayaran kartu kredit ya...,NaN,NaN,NaN,NaN,Inquiry,False
9,9151919416110000060,Inquiry,Nasabah menanyakan status penghapusan biaya an...,NaN,NaN,NaN,NaN,NaN,False
